In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))



In [2]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout,Flatten,Conv2D,MaxPooling2D
import tensorflow as tf
from tensorflow.keras.preprocessing import image

2024-05-02 20:54:15.976403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 20:54:15.976512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 20:54:16.141746: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
test_dir="../input/dogs-cats-images/dog vs cat/dataset/test_set" #getting our testing data
train_dir="../input/dogs-cats-images/dog vs cat/dataset/training_set" #getting our training data

train_dir_cats = train_dir + '/cats' #storing the cats training images
train_dir_dogs = train_dir + '/dogs' #storing the dogs training images
test_dir_cats = test_dir + '/cats' #storing the cats testing images
test_dir_dogs = test_dir + '/dogs' #storing the dogs testing images

In [5]:
data_generator=ImageDataGenerator(rescale=1/250 ,zoom_range=0.2)

In [6]:
batch_size=10
X_train=data_generator.flow_from_directory(
batch_size=batch_size,
directory=train_dir,
target_size=(200,200),
class_mode='binary')

X_test=data_generator.flow_from_directory(
batch_size=batch_size,
directory=test_dir,
class_mode='binary',
target_size=(200,200))

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
model=Sequential([
    Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = X_train.image_shape),
    MaxPooling2D((2,2)),
    Dropout(rate = 0.3),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(rate = 0.15),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(rate=0.15),
    Dense(64,activation='relu'),
    Dense(units = len(set(X_train.classes)), activation = 'softmax')
    
])



/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [8]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 147456)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    18,874,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,902,274 (72.11 MB)

 Trainable params: 18,902,274 (72.11 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
fitted_model=model.fit(
X_train,
validation_data=X_test,
    epochs=2,
    
)

Epoch 1/2
800/800 ━━━━━━━━━━━━━━━━━━━━ 432s 539ms/step - accuracy: 0.4990 - loss: 0.6933 - val_accuracy: 0.5025 - val_loss: 0.6880
Epoch 2/2
800/800 ━━━━━━━━━━━━━━━━━━━━ 416s 519ms/step - accuracy: 0.5439 - loss: 0.6892 - val_accuracy: 0.5655 - val_loss: 0.6750


In [12]:
def testing_image(image_directory): #testing out our model
    test_image = image.load_img(image_directory, target_size = (200, 200))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(x = test_image)
    print(result)
    if result[0][0]  == 1:
        prediction = 'Dog'
    else:
        prediction = 'Cat'
    return prediction

In [13]:

print(testing_image(test_dir + '/cats/cat.4003.jpg'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
[[0.00164169 0.9983583 ]]
Cat
